<a href="https://colab.research.google.com/github/UchidaYasuto/DH/blob/main/Complete_RAG_JTwp_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Building a Retrieval-Augmented Generation (RAG) for querying humanities knowledge**

## **使用するデータとツール**

### **データ**
- **ソース**  
  **[情報通信白書（通信白書）]**(https://www.soumu.go.jp/johotsusintokei/whitepaper/)  
  [総務省](https://www.soumu.go.jp/) が情報通信の分野における産業の現況や政策の動向などを取りまとめ、年次で刊行。同白書の内容には、IT（情報技術）およびICT（情報通信技術）に関する政府の取り組み、成長戦略、諸政策とその進捗状況、国内外の業界の動向、今後の方針や展望、各種調査資料などが含まれる。1973年に「通信白書」として初めて刊行されてから毎年刊行されており、2025年に発表された「令和7年版情報通信白書」で通算53版を数える。  
  この[HTML版](https://www.soumu.go.jp/johotsusintokei/whitepaper/)をソース文書として利用
- **埋め込み方法（Embedding method）**  
  [OpenAIEmbeddings](https://platform.openai.com/docs/guides/embeddings) のデフォルトモデルを使用し、文書をベクトルに変換
- **ベクトルデータベース（Vector DB）**  
  ベクトルの保存：軽量でオープンソースの埋め込みデータベース [Chroma](https://www.trychroma.com/) を使用
- **検索方法（Retrieval method）**  
  検索：Chroma のデフォルトリトリーバを使用

### **モデル**
- **リトリーバモデル（Retriever model）**  
  検索： OpenAIEmbedding のデフォルトモデルを使用
- **生成モデル（Generative model）**  
  生成：OpenAI の `gpt-4o-mini` モデルを使用

### **実装**
- [RAGシステム](https://en.wikipedia.org/wiki/Retrieval-augmented_generation) の構築に [LangChain](https://ja.wikipedia.org/wiki/LangChain) を利用  
LangChain：大規模言語モデル（LLM）を活用したアプリケーションを構築するフレームワーク. モデルと外部ツール・データベースをモジュール的に組み合わせる

---
### **RAGシステム構築ツール**

1. **`langchain`** ：言語モデルを用いたアプリケーションを構築するための主要パッケージ。データ処理、情報検索、応答生成の流れを整理して構築可能

2. **`langchain-community`**： LangChain コミュニティによる拡張ツールや追加機能を含むパッケージ。LangChainの機能をさらに強化

3. **`langchain_text_splitters`**：大きな文書を小さなチャンクに分割するツール。RAGシステムでは長文を処理しやすくするために分割して利用

4. **`langchain_chroma`**：LangChain と **Chroma**（ベクトルデータベース）を接続するためのパッケージ。質問に答える際、最も関連性の高い情報を検索・取得する中核的な役割を担う

In [ ]:
# Google Colab 上で LangChain を正しくインストールするために、システムのエンコーディングを UTF-8 に設定
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Additional requirement to use LLM from OpenAI and other LangChain components
!pip install -qU langchain-openai langchain langchain-community langchain_text_splitters langchain_chroma langchain-core

# Additional requirement to use open-source LLM from hugging face community
!pip install -q torch transformers accelerate sentence-transformers faiss-cpu

---
[LangChain](https://ja.wikipedia.org/wiki/LangChain) を用いて **大規模言語モデルLLM** を利用  
[OpenAI](https://openai.com/) の有料LLM（OpenAI GPT-4o）を使用して、埋め込み・検索・テキスト生成などのタスクを実行  
以下の手順で、APIキーを入力し、モデルを初期化

In [ ]:
# Non-free LLM, OpenAI gpt-4o
import getpass
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", temperature = 0.0)  # for embedding, retriever and generator

---
### **全体の流れ**
以下、3つのステップを通して、RAG構築を進めていく
1. Webソースからのコンテンツ取得･データファイル構築
2. ベクトルデータベースの構築
3. RAGの構築と運用



---
# **１　Webソースからのコンテンツ取得･データファイル構築**

### Source: [総務省](https://www.soumu.go.jp/) [『情報通信白書（通信白書）』昭和48年度版～令和7年度版：計53年分](https://www.soumu.go.jp/johotsusintokei/whitepaper/)



LangChainを用いて、テキスト（TXT）形式の文書を直接処理

* 各年次の TOC（Table Of Contents：目次リンク一覧） から本文HTMLへのリンクを全て取得

* `requests`ライブラリを使用して、上記Sourceのリンクからウェブ内のHTMLコンテンツを取得
* `BeautifulSoup`を使用してHTMLを解析、本文を抽出
* 本文ｎスクリプトをクリーニング：全リンク（`href`属性を持つ`<a>`タグ）を抽出して処理、アンカーや`mailto:`リンクなどの不要なリンクや不要な表記（ページ数など）を除外、相対URLを絶対URLに変換、余計な空行･空白の削除、不要な文の削除（「〜形式のファイルは こちら」、「〜に戻る / 〜に進む」など）
* 指定年度ごとに Document を作成、各年度のテキストを結合して53年分の「情報通信白書（通信白書）」を1つのテキストファイルに保存

In [ ]:
import re
from pathlib import Path
import requests
from typing import List, Dict
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
# ===== YEAR_MAP（和暦→西暦） =====
YEAR_MAP = {}

# 昭和: s01 = 1926
showa_start = 1926
for i in range(1, 65):  # s01 ~ s64 くらいまで
    era = f"s{i:02d}"
    western = showa_start + i - 1
    YEAR_MAP[era] = str(western)

# 平成: h01 = 1989
heisei_start = 1989
for i in range(1, 32):  # h01 ~ h31
    era = f"h{i:02d}"
    western = heisei_start + i - 1
    YEAR_MAP[era] = str(western)

# 令和: r01 = 2019
reiwa_start = 2019
for i in range(1, 10):  # r01 ~ r09 くらいまで
    era = f"r{i:02d}"
    western = reiwa_start + i - 1
    YEAR_MAP[era] = str(western)

In [ ]:
# 各年次の TOC（Table Of Contents：目次リンク一覧） から本文HTMLへのリンクを全て取得
# Case１：平成24年 (h24)、平成22年 (h22)、昭和48年～平成21年: /ja/{year}/index.html がTOCを含むため、そこから /ja/{year}/html/*.html へのリンクを取得
# Case２：平成23年（h23）以降（h24を除く）：原則として nb000000.html の <noscript> から取得

from typing import List, Dict

def get_toc_links(year: str) -> List[Dict[str, str]]:
    links: List[Dict[str, str]] = []

    # Case 1: 平成24年および平成22年以前
    if (
        year in ["h24", "h22", "h15"]
        or (year.startswith("h") and 1 <= int(year[1:]) <= 21)
        or (year.startswith("s") and 48 <= int(year[1:]) <= 63)
    ):
        toc_url = f"https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/{year}/index.html"
        print(f"[INFO] ({year} index.html) TOC: {toc_url}")
        html = fetch_and_decode(toc_url)
        if not html:
            return links

        soup = BeautifulSoup(html, "html.parser")
        container = soup.body or soup
        base_prefixes = [
            f"https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/{year}/",
            f"https://www.soumu.go.jp/johotsusintokei/whitepaper/{year}/",
        ]

        for a in container.find_all("a", href=True):
            href = a["href"]
            full = urljoin(toc_url, href)
            low = full.lower()
            if not any(full.startswith(p) for p in base_prefixes):
                continue
            if not (low.endswith(".html") or low.endswith(".htm")):
                continue
            if full == toc_url:
                continue
            title = a.get_text(strip=True)
            links.append({"title": title, "url": full})

    # Case 2: 平成23年および平成25年以降
    else:
        toc_url = "https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/{year}/html/nb000000.html".format(year=year)
        print(f"[INFO] ({year} nb000000.html) TOC: {toc_url}")
        html = fetch_and_decode(toc_url)
        if not html:
            return links

        soup = BeautifulSoup(html, "html.parser")
        left_sub = soup.find(id="left_sub")
        noscript = left_sub.find("noscript") if left_sub else soup.find("noscript")
        if noscript is None:
            print(f"[WARN] {year}: <noscript> not found, falling back to body scan.")
            scan_root = soup.body or soup
            base_url = f"https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/{year}/"
            for a in scan_root.find_all("a", href=True):
                href = a["href"]
                full = urljoin(toc_url, href)
                if full.startswith(base_url) and full.lower().endswith((".html", ".htm")):
                    title = a.get_text(strip=True)
                    links.append({"title": title, "url": full})
        else:
            for a in noscript.find_all("a", href=True):
                title = a.get_text(strip=True)
                full = urljoin(toc_url, a["href"])
                ok = (
                    f"/johotsusintokei/whitepaper/ja/{year}/html/" in full
                    and full.startswith(f"https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/{year}/html/")
                )
                if ok:
                    links.append({"title": title, "url": full})

            # nb000000 自身も含める
            if not any(item["url"] == toc_url for item in links):
                nb = noscript.find("a", href=f"/johotsusintokei/whitepaper/ja/{year}/html/nb000000.html")
                title = nb.get_text(strip=True) if nb else "概要（nb000000）"
                links.insert(0, {"title": title, "url": toc_url})

    # 重複排除
    seen = set()
    uniq = []
    for it in links:
        if it["url"] not in seen:
            seen.add(it["url"])
            uniq.append(it)

    print(f"[INFO] {year}: {len(uniq)} links")
    return uniq

In [ ]:
# URLからコンテンツを取得、テキストとしてディコード（HTTP 取得：リトライ & ディコード）
def fetch_and_decode(url: str) -> str:
    try:
        resp = requests.get(url)
        resp.raise_for_status()

        # ---- ディコード ----
        html = None
        encodings_to_try = [resp.apparent_encoding, "utf-8", "shift_jis", "euc-jp"]
        for enc in encodings_to_try:
            if not enc:
                continue
            try:
                html = resp.content.decode(enc, errors="strict")
                # print(f"[INFO] Decoded with {enc}")
                break
            except Exception:
                continue

        if html is None:
            for enc in encodings_to_try:
                if not enc:
                    continue
                try:
                    html = resp.content.decode(enc, errors="ignore")
                    # print(f"[INFO] Decoded with {enc} (ignore)")
                    break
                except Exception:
                    continue

        if html is None:
            print(f"[ERROR] Failed to decode: {url}")
            return None

        return html

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Failed to fetch {url}: {e}")
        return None

In [ ]:
# 本文抽出（A/Bパターン）
def extract_main_text_from_html(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()

    # パターンA: id="contents"
    main = soup.find("div", id="contents")
    # パターンB: id="tdrcont" → 親の td.tdr
    if main is None:
        anchor = soup.find(id="tdrcont")
        if anchor:
            td = anchor.find_parent("td", class_="tdr")
            main = td if td else anchor.parent

    # フォールバック
    if main is None:
        main = soup.body if soup.body else soup

    raw_text = main.get_text(separator="\n")
    return clean_text(raw_text)

In [ ]:
# クリーニング
REMOVE_LINE_PATTERNS = [
    # 「〜形式のファイルは こちら」（前後に改行、単独行）
    re.compile(r"^\s*(テキスト|Excel|PDF)形式のファイルは\s*（?こちら）?\s*こちら\s*$", re.IGNORECASE),
    re.compile(r"^\s*(テキスト|Excel|PDF)形式のファイルは\s*こちら\s*$"),
    # 「〜に戻る / 〜に進む」（単独行）
    re.compile(r"^\s*.+に戻る\s*$"),
    re.compile(r"^\s*.+に進む\s*$"),
    # こちらのみの行（安全のため極端な短文を削除）
    re.compile(r"^\s*こちら\s*$"),
]

def clean_text(text: str) -> str:
    lines = text.splitlines()
    paragraphs, current = [], []

    def flush():
        nonlocal current, paragraphs
        if current:
            paragraphs.append("".join(current))
            current = []

    for line in lines:
        raw = line
        s = raw.strip()

        # 単独行の削除条件
        if s:
            for pat in REMOVE_LINE_PATTERNS:
                if pat.match(s):
                    s = ""  # ドロップ
                    break
        if s == "":
            flush()
            continue

        # 段落内マージ（英数連結のみスペース挿入）
        if not current:
            current.append(s)
        else:
            prev = current[-1]
            joiner = " " if (prev and s and prev[-1].isalnum() and s[0].isalnum()) else ""
            current.append(joiner + s)

    flush()
    text = "\n\n".join(paragraphs)
    text = re.sub(r"[ \t]{2,}", " ", text)
    return text

In [ ]:
# 年度ごとの Document を作成
"""
ある和暦（例: 'h15'）について、
 - get_toc_links でリンク一覧を取得
 - 各ページを取得して本文抽出
 - YEAR/TITLE/URL をヘッダーにしたテキストを連結して 1 つの文字列を返す
"""
def build_year_corpus_text(year_era: str) -> str:
    year_western = YEAR_MAP.get(year_era)
    if not year_western:
        print(f"[ERROR] Unknown year era: {year_era}. Skipping.")
        return ""

    links = get_toc_links(year_era)
    parts = []

    for i, item in enumerate(links, start=1):
        url = item["url"]
        title = item["title"].strip()
        print(f"[INFO] {year_era} ({year_western}) [{i}/{len(links)}] 取得中: {url}")

        try:
            resp = requests.get(url)
            resp.raise_for_status()

            # ---- ディコード ----
            html = None
            encodings_to_try = [resp.apparent_encoding, "utf-8", "shift_jis", "euc-jp"]
            for enc in encodings_to_try:
                if not enc:
                    continue
                try:
                    html = resp.content.decode(enc, errors="strict")
                    print(f"[INFO] Decoded with {enc}")
                    break
                except Exception:
                    continue

            if html is None:
                for enc in encodings_to_try:
                    if not enc:
                        continue
                    try:
                        html = resp.content.decode(enc, errors="ignore")
                        print(f"[INFO] Decoded with {enc} (ignore)")
                        break
                    except Exception:
                        continue

            if html is None:
                print(f"[ERROR] Failed to decode: {url}")
                continue

            # ---- 本文抽出（既存の関数を利用）----
            page_text = extract_main_text_from_html(html).strip()
            if not page_text:
                print(f"[INFO] {url} は本文ほぼ空なのでスキップ")
                continue

            # ヘッダー行を付けて追加
            header = f"===== YEAR {year_western} | TITLE: {title} | URL: {url} =====\n\n"
            parts.append(header + page_text + "\n")

        except requests.exceptions.RequestException as e:
            print(f"[ERROR] Failed to fetch {url}: {e}")
            continue

    year_text = "\n\n".join(parts)
    print(f"[INFO] {year_era} ({year_western}): 文字数 = {len(year_text)}")
    return year_text

In [ ]:
# 指定した和暦（例: ["r07","r01","h24",...,"s48"]）について各年度のテキストを結合、1つのファイルを作成･保存
def build_multi_year_corpus_text(
    years_era: list[str],
    out_path: Path,
) -> str:
    all_parts = []

    for year_era in years_era:
        text = build_year_corpus_text(year_era)
        if text:
            all_parts.append(text)

    full_text = "\n\n".join(all_parts)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_text(full_text, encoding="utf-8")

    print(f"[INFO] Combined corpus saved to: {out_path}  （文字数: {len(full_text)}）")
    return full_text

In [ ]:
# 実行：53年分の情報通信白書を1つのテキストファイルに
target_years = ["s48", "s49", "s50","s51","s52","s53","s54","s55", "s56", "s57", "s58", "s59", "s60","s61","s62","s63","h01","h02","h03","h04","h05", "h06", "h07", "h08", "h09", "h10","h11","h12","h13","h14","h15", "h16", "h17", "h18", "h19", "h20","h21","h22","h23","h24","h25", "h26", "h27", "h28", "h29", "h30", "r01", "r02", "r03", "r04", "r05", "r06", "r07"]

# Convert era years to Western years for min/max calculation for the filename
western_years_for_filename = sorted([int(YEAR_MAP[y_e]) for y_e in target_years if y_e in YEAR_MAP])
if western_years_for_filename:
    min_year = min(western_years_for_filename)
    max_year = max(western_years_for_filename)
    new_filename = f"Joho-Tsushin_whitepaper_{min_year}-{max_year}.txt"
else:
    new_filename = "Joho-Tsushin_whitepaper_combined.txt" # Fallback if no years

output_file = Path(new_filename)
combined_text = build_multi_year_corpus_text(target_years, output_file)

In [ ]:
# 作成したテキストファイルをダウンロード
from google.colab import files
import os

# The actual combined file generated by build_multi_year_corpus_text
# from cell xt6GNFGD08Xa is named 'whitepaper_r07_r01_h24_h22_h15_s63_s48_combined.txt'
# and is saved in the current working directory (/content/).
actual_combined_filename = "whitepaper_r07_r01_h24_h22_h15_s63_s48_combined.txt"

# Only attempt to download the actual combined file
file_paths = [actual_combined_filename]

for file_path in file_paths:
    try:
        files.download(file_path)
        print(f"Downloaded: {file_path}")
    except FileNotFoundError:
        print(f"File not found for download: {file_path}")
    except Exception as e:
        print(f"An error occurred during download of {file_path}: {e}")

Webのリンクから抽出･作成したテキストファイル  
* [Joho-Tsushin_whitepaper_1973-2025.txt](https://drive.google.com/file/d/1cdGCh1VhyhS695MitsK6gX4HFLrqFyFj/view?usp=drive_link)


---
# **２　ベクトルデータベースの構築**

### **データ前処理･メタデータ付与･チャンク化**

In [ ]:
from pathlib import Path
from langchain_core.documents import Document
import re

# 上で作成された combined テキストファイルを読み込み、
# ヘッダー行 ===== YEAR 1993 | TITLE: ... | URL: ... ===== ごとに区切って Document を作る。

def load_corpus_file_to_documents(corpus_path: Path) -> list[Document]:
    text = corpus_path.read_text(encoding="utf-8")

    # ヘッダー行を正規表現で探す
    pattern = re.compile(
        r"^===== YEAR (?P<year>\d{4}) \| TITLE: (?P<title>.*?) \| URL: (?P<url>\S+) =====\s*$",
        re.MULTILINE,
    )

    docs: list[Document] = []
    matches = list(pattern.finditer(text))

    for i, m in enumerate(matches):
        # None の可能性を考慮して、strip() を呼び出す前にチェックを追加
        year = m.group("year")
        title = m.group("title")
        url = m.group("url")

        year = year.strip() if year else ""
        title = title.strip() if title else ""
        url = url.strip() if url else ""

        start = m.end()  # このヘッダーの直後
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        body = text[start:end].strip()
        if not body:
            continue

        doc = Document(
            page_content=body,
            metadata={
                "year": year,
                "title": title,
                "url": url,
                "source": url,  # retriever からも参照しやすいように
                "block_index": i,  # 何番目のヘッダーか（任意）
            },
        )
        docs.append(doc)

    print(f"[INFO] Parsed {len(docs)} Document blocks from: {corpus_path}")
    return docs

LangChainの`RecursiveCharacterTextSplitter`を使用して、大きな文書をより小さく管理しやすいチャンクに分割

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Document をチャンク分割(メタデータは自動的に継承)

def chunk_documents_with_meta(
    docs: list[Document],
    # chunk_size: int = 750,
    # chunk_overlap: int = 100,
    chunk_size=2000,          # 文字数で 2000 前後
    chunk_overlap=200,        # 前後に少し重ねる
) -> list[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", "。", "、", " "]  # 日本語用に少し優しめ
    )
    chunks = splitter.split_documents(docs)
    print(f"[INFO] Chunked into {len(chunks)} documents.")
    return chunks

# まず、コーパスファイルを読み込む
corpus_path = Path('Joho-Tsushin_whitepaper_1973-2025.txt')
docs = load_corpus_file_to_documents(corpus_path)

# 次に、ドキュメントをチャンクに分割
chunks = chunk_documents_with_meta(docs)

# chunks

### **データベース(the knowledge base)の用意**

**（１）新規データベース構築**
* 構築したデータベースのダウンロード
* 構築したデータベースをそのまま使用

**（２）既存データベース再利用**  
* Googleドライブからの読み込み
* zipファイルを解凍して再ロード



**Embedding model**（埋め込みモデル）  
テキストデータを数値ベクトルに変換するための機械学習モデル  
単語や文章などのテキストを高次元の数値ベクトルに変換

### （１）新規データベース構築
OpenAIの有料埋め込みモデルと、埋め込みを保存する軽量なオープンソース**Chroma**を使用し、ベクトルデータベース（DB）を構築

In [ ]:
# ①スピード重視：TPM（1分あたりのトークン制限）リスク高
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_function = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=20,   # ← バッチの大きさを制御
)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory="./chroma_db",
)

In [ ]:
# ②安全性重視
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import time

embedding_function = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=5,
)

vectorstore = Chroma(
    embedding_function=embedding_function,
    persist_directory="./chroma_db"
)

for i in range(0, len(chunks), 50):     # 50件ずつ安全に処理
    batch = chunks[i:i+50]
    vectorstore.add_documents(batch)
    print(f"{i+len(batch)} / {len(chunks)} done")
    time.sleep(2)    # 少し待つ（TPM回避）

In [ ]:
# ③より安全に（時間はかかる：45分ほど）
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from tqdm import tqdm
import time
import random

# 1. Embedding function
embedding_function = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=5,  # バッチ小さくして安全に
)

# 2. Create Chroma DB
vectorstore = Chroma(
    embedding_function=embedding_function,
    persist_directory="./chroma_db"
)

# 3. バッチ埋め込み設定
BATCH_SIZE = 50

# 4. 進捗バー付き安全埋め込み
for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc="Embedding batches"):
    batch = chunks[i:i+BATCH_SIZE]

    while True:
        try:
            vectorstore.add_documents(batch)      # 埋め込み＋保存
            break                                 # 成功したら次へ

        except Exception as e:
            print(f"\n Error at batch {i}: {e}")

            # RateLimit 対策（指数バックオフ）
            wait = random.uniform(5, 10)
            print(f"Waiting {wait:.1f} sec before retry...")
            time.sleep(wait)

    time.sleep(0.5)  # 軽めのスロットル

print("\n Embedding completed successfully!")

### 構築したデータベースのダウンロード

In [ ]:
# ダウンロード①
# データベースディレクトリをzipファイルに圧縮、ダウンロード
import shutil
import os

# データベースディレクトリのパス
db_directory = "./chroma_db"
# 圧縮後のzipファイル名
zip_filename = "chroma_db.zip"

# ディレクトリが存在することを確認
if os.path.exists(db_directory):
    # ディレクトリをzipファイルに圧縮
    shutil.make_archive("chroma_db", "zip", root_dir=".", base_dir="chroma_db")
    print(f"'{db_directory}' が '{zip_filename}' に圧縮されました。")
else:
    print(f"エラー: ディレクトリ '{db_directory}' が見つかりません。")

from google.colab import files
files.download("chroma_db.zip")

In [ ]:
# ダウンロード②：TPM（1分あたりのトークン）制限のリスク
# データベースディレクトリをzip化、ファイルをダウンロード
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding_function = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=5,
)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory="./chroma_db",
)

# zip を作る（シェル版の方がトラブル少ない）
!zip -r chroma_db.zip chroma_db

from google.colab import files
files.download("chroma_db.zip")

### 構築したデータベースをそのまま使用

In [ ]:
# 埋め込みモデルの設定とベクトルデータベースの初期化

# from langchain_openai import OpenAIEmbeddings
# from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=5,
)

vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./chroma_db"
)

### （２）既存データベース再利用

すでに作成･準備された既存の**Chroma**データベースを再利用、埋め込みを直接読み込む

### ①Googleドライブから Choma_db を読み込む

* [Choma_db](https://drive.google.com/drive/folders/1cFD3QW7_fHty7Y5XEslH9WM1LwP8aJvR?usp=sharing)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# download existing Chroma DB from google drive
!gdown --folder https://drive.google.com/drive/folders/1cFD3QW7_fHty7Y5XEslH9WM1LwP8aJvR?usp=sharing

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# ① embedding_function を定義
embedding_function = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=5,   # ← このプロジェクトの推奨設定
)

# ② その後、保存済みの Chroma_DB を読み込む
vectorstore = Chroma(
    embedding_function=embedding_function,
    persist_directory="./chroma_db"  # 既存DBフォルダ
)

### ②chroma_db.zipを解凍して再ロード

* [chroma_db.zip](https://drive.google.com/file/d/1Gl5Erm3EAFSn7xuDHEPlXyngNInsfrvt/view?usp=drive_link)


In [ ]:
# 再アップロード方法を選択
# ①PC内のローカルフォルダーから選択
from google.colab import files
uploaded = files.upload()              # chroma_db.zip を選ぶ

!unzip -o chroma_db.zip                # ここでエラーが出ないか確認

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=20,
)

vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./chroma_db"
)

---
# **３　RAGの構築と運用**

### **ベクトルデータベース Choma_db の用意**

すでに作成･準備された既存の**Chroma**データベースを再利用、埋め込みを直接読み込む方針をとる

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# download existing Chroma DB from google drive
!gdown --folder https://drive.google.com/drive/folders/1cFD3QW7_fHty7Y5XEslH9WM1LwP8aJvR?usp=sharing

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# ① embedding_function を定義
embedding_function = OpenAIEmbeddings(
    model="text-embedding-3-large",
    chunk_size=5,   # ← このプロジェクトの推奨設定
)

# ② その後、保存済みの Chroma_DB を読み込む
vectorstore = Chroma(
    embedding_function=embedding_function,
    persist_directory="./chroma_db"  # 既存DBフォルダ
)

### **Retriever**

LangChainでデータベース（chroma_db）にアクセスするリトリーバーを設定 ⇒以下の２つの方法を併用
*   cosine 類似度（similarity）：「質問と一番似ているチャンクを上から順に取ってくる」シンプルな方式
*   MMR（max marginal relevance, mmr）：「質問に近い＋お互いに似すぎていないチャンクを選ぶ」方式

In [ ]:
# Retriever using vector DB
retriever = vectorstore.as_retriever()

retriever_sim = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10},
)

retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 20,
        "lambda_mult": 0.5,  # 0〜1で調整（0寄り → 多様性重視）
    },
)

## **Prompt (question + context) template**

**プロンプト**：LLMに与えられる最初の入力または質問　モデルが応答すべきタスクやクエリを定義
1. **ユーザーのクエリ**：ユーザーの主要な質問またはタスク
2. **コンテキストとして取得された文書**：ベクトルデータベースからリトリーバーによって取得された追加情報、LLMがクエリのコンテキストを理解し正確な応答を生成するのに役立つ

*RAGにおけるプロンプトの動作*：
1. **ユーザークエリ**： ユーザーが質問をするか、入力を提供（メインプロンプト）   
2. **検索**： リトリーバーがベクターデータベースで関連文書を検索し、クエリに最も類似した文書のセットを返す   
3. **拡張プロンプト**： 検索によって取得された文書はユーザークエリと組み合わされ、**拡張プロンプト**を作成。ユーザーの入力と関連するコンテキストの両方をLLMに提供することで、詳細かつ正確な応答を生成しやすくする   
4. **応答生成**： LLMが拡張プロンプトを処理、ユーザーのクエリと取得された文書の追加コンテキストの両方に基づいて応答を生成

In [ ]:
# PromptTemplateクラスをLLM用のカスタムプロンプトテンプレートを作成するために使用
from langchain_core.prompts import PromptTemplate

# プロンプトテンプレート（`<||>`はセクションを、`{}`はプレースホルダーを表す）：
# `<|system|>`: システムの役割を定義し、LLMへの一般的な指示を提供
# `{context}`: ナレッジベースから取得したコンテキストのプレースホルダー
#   ユーザーのクエリに関連する情報で動的に置換される
# `<|user|>`: ユーザーの入力（質問）を表す
# {question}: ユーザーの実際の質問に置換されるプレースホルダー
# `<|assistant|>`: 言語モデル（アシスタント）が応答を生成する箇所
# コンテキストと質問の両方に基づいて応答を生成

prompt_template = """
<|system|>
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
keep the answer as concise as possible.
Use markdown formatting when displaying code.
Emphasis should be used to terminologies.
Always say "thanks for asking!" at the end of the answer.

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

"""

#  Create the PromptTemplate Instance
prompt = PromptTemplate(
    input_variables=[
        "context",
        "question",
        "sources"
        ],
    template=prompt_template,
)

## **Output formatter**

**Document (context) formatter**
* 処理された文書を単一の文字列にフォーマットする関数を定義
* 表示やさらなる処理が簡単になり、複数の文書チャンクの内容を出力または可視化したい場合に便利

**Source information formatter**
* 取得された文書からソース（メタデータ）を抽出してフォーマットする関数を定義
* ソース情報（URLや文書タイトルなど）をコンテキストの一部として言語モデル（LLM）に渡したい場合に便利
Extract sources (e.g., URLs or document titles) from metadata  
Join the sources into a single string to pass to the LLM

**Answer formatter**
* Jupyterノートブックや、IPythonの表示機能をサポートする環境内で、出力（回答）をきれいにフォーマットされたMarkdown形式で表示


In [ ]:
# Document (context) formatter
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Source information formatter
def format_sources(docs):
    sources = [doc.metadata.get('source', 'Unknown Source') for doc in docs]
    return "Sources:\n" + "\n".join(sources)

# Answer formatter
from IPython.display import Markdown, display, HTML

def display_answer(answer):
    # Wrap the answer string directly in an HTML div with CSS to force word wrapping
    # Using 'word-break: break-word' for more aggressive breaking if needed
    wrapped_html = f"<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>{answer}</div>"
    display(HTML(wrapped_html))

## **Overall workflow by combining the above components**

以下のコンポーネントを組み合わせ、RAGシステムを使用したワークフローを構築
* Sources：Database (documents)
* Question
* Retrieval(search)
* Context(Docs merged)
* Answer(LLM Gen)  

**RAGchain**：LangChainフレームワーク内の専門的なコンポーネント  
・検索と生成を1つのシームレスなワークフローで組み合わせるプロセスを効率化するように設計  
・RAGchainが、このプロセスをより効率的にする方法は以下のとおり：
- **リトリーバーとジェネレーターの統合**：リトリーバー（関連文書を探索･取得）とジェネレーター（取得された文書に基づいて最終的な応答を作成）の両方を1つの統合されたワークフローに組み合わせる
- **コンテキストを考慮した生成**：検索を生成プロセスに直接統合し、生成される応答がより正確で文脈的に関連性の高いものにする
- **簡略化されたワークフロー**：検索と生成の個別のステップを手動で処理する必要がなく、フレームワークがプロセス全体を処理するため、実装と保守がより簡単に

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm_chain = prompt | llm | StrOutputParser()

# similarity 用 RAG チェーン
rag_chain_sim = (
    {
        "context": retriever_sim | format_docs,     # ← ここだけ sim retriever
        "sources": retriever_sim | format_sources,
        "question": RunnablePassthrough(),
    }
    | llm_chain
)

# MMR 用 RAG チェーン
rag_chain_mmr = (
    {
        "context": retriever_mmr | format_docs,     # ← ここだけ mmr retriever
        "sources": retriever_mmr | format_sources,
        "question": RunnablePassthrough(),
    }
    | llm_chain
)

---
## **Q&Aのフォーマット**

**１）RAGの回答**：以下の２つの方法を併用し、検索結果を比較
*   **Similarity RAG**（cosine 類似度）：「質問と一番似ているチャンクを上から順に取ってくる」シンプルな方式
*   **MMR RAG**（MMR:Max Marginal Relevance）：「質問に近い＋相互に似すぎていないチャンクを選ぶ」方式

**２）LLMの回答との比較**

**３）RAGのソース情報の提示**
*   **RAGの回答のソースリスト**：項目タイトル、URL
*   **RAGの回答のソース詳細**：年次、項目タイトル、URL、内容

---

## **＜質問はこちらに＞**

In [ ]:
# question
query = "　？"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   **Similarity RAG**（cosine 類似度）：質問と一番似ているチャンクを上から順に５つ選択
*   **MMR RAG**（MMR:Max Marginal Relevance）：質問に近い＋相互に似すぎていないチャンクを10個選択

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---
# **Showcase**

---
## Q1　数値把握／状況説明問題（テレトピア政策の進捗状況）

外部知識（昭和63年版 情報白書の1-1-3 地域情報化推進政策の現状）に関する以下の文脈について、RAGシステムに問い合わせる：

> 郵政省では，テレトピア事業や民活法施設の整備事業等を通じて，地域に密着した情報通信を中心とした情報化を推進している。ここでは，情報通信を中心とした地域の情報化推進政策の現状について概観する。  
> （1）テレトピアの推進  
**テレトピア計画は，ＣＡＴＶやビデオテックス等のニューメディアをモデル都市に集中的に導入することにより，地域の情報化を促進し，それぞれの地域の情報通信の核となる基盤を整備するもの**である。
テレトピアは，現在，**63地域が指定**され，全国で260のシステム構築予定に対し，**62年度末現在，既に92システムが運用を開始**している。また，これらのシステムの事業主体として41の第三セクターが設立されている。
運用システムに導入されているメディアとしては，データ通信，ビデオテックス及びＣＡＴＶが中心であり，全体の8割を占めている。


## **＜質問はこちらに＞**

In [ ]:
# question
query = "昭和62年時点で、テレトピアの指定地域数やシステムの整備状況はどうなっていますか。"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   **Similarity RAG**（cosine 類似度）：質問と一番似ているチャンクを上から順に５つ選択
*   **MMR RAG**（MMR:Max Marginal Relevance）：質問に近い＋相互に似すぎていないチャンクを10個選択

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---

---
## Q2　語彙･概念の説明問題（「テレトピア」の説明）

## **＜質問はこちらに＞**

In [ ]:
# question2
query = "テレトピアとは何ですか。"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   **Similarity RAG**（cosine 類似度）：質問と一番似ているチャンクを上から順に５つ選択
*   **MMR RAG**（MMR:Max Marginal Relevance）：質問に近い＋相互に似すぎていないチャンクを10個選択

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---

---
## Q3　数値把握･説明問題（情報化指数の国際比較 1970年時点）

外部知識（『昭和48年版 情報白書』情報化指数の国際比較）に関する以下の文脈について、RAGシステムに問い合わせる：

> 情報化指数により国際比較を行うと，1970年現在で日本を100とした場合，米国199，英国108，西独101，フランス96となり，我が国は総体的には西欧並みであるが，米国には遠く及ばない状態である。
　特に我が国が他国に比べて大きい値を示すのは，1人当たり年間通話度数（日本382回，米国779，英国195，西独166，フランス105），100人当たり1日平均新聞発行部数（日本51.1部，米国30.2，英国46.3，西独31.9，フランス23.8），人口密度（日本280人／ｋｍ2，米国22，英国228，西独240，フランス93），100人当たりの大学在学者数（日本1.56人，米国3.82、英国0.62，西独0.74，フランス1.20）である。
　反対に日本が他国に比べて小さい値を示す項目は1万人当たり年間書籍発行点数（日本3.03点，米国3.88，英国5.97，西独7.69，フランス4.50），1万人当たり電子計算機台数（日本0.65台，米国3.06，英国1.06，西独1.08，フランス0.88）である。これをみると，1万人当たり書籍発行点数が他の国に比べて非常に小さいことのほか，人的要素による情報化（通話度数，人口密度，大学在学者数）は高いが，テレビ，電子計算機などのいわば情報装備率ともいうべきものが低いことが特徴的である


In [ ]:
# question
query = "1970年時点における日本の情報化の進捗状況について，情報化指数により国際比較を行うと，どのような状況になっていますか。"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   **Similarity RAG**（cosine 類似度）：質問と一番似ているチャンクを上から順に５つ選択
*   **MMR RAG**（MMR:Max Marginal Relevance）：質問に近い＋相互に似すぎていないチャンクを10個選択

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---

---
## Q4　数値把握／状況説明問題（昭和48年における年代別テレビ視聴時間の特徴）

外部知識（『昭和48年版 情報白書』テレビ視聴時間）に関する以下の文脈について、RAGシステムに問い合わせを試みる：

> テレビ視聴時間を年代別にみると，年代層によって変化の状況にかなりの相違がある。特に，20代がほとんど変化していないのに対し，10代が若干減少気味であり，逆に50代，60代は大幅に増加している。これを詳細にみると，10代は40年の2時間18分に対して45年は2時間9分で，9分減少しており，一方増加が顕著な60代は，40年の3時間16分に対して45年は4時間1分となり，45分増えている。このように年代層によってテレビ視聴時間の増減傾向にかなりの差があるが，ＮＨＫ放送世論調査所が行った「全国意向調査」によると，男女とも若い世代ほどテレビ放送番組に対する選択意識が強く，「見たい番組がある時に見る」比率が高いのに対して，高年齢層になるに従って「見るのが習慣になっている」比率が高く，極めて対照的である



In [ ]:
# question
query = "昭和48年におけるテレビ視聴時間は、年代別にみてどのような特徴がありましたか。"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   Similarity RAG（cosine 類似度）：「質問と一番似ているチャンクを上から順に取ってくる」シンプルな方式
*   MMR RAG（MMR:Max Marginal Relevance）：「質問に近い＋相互に似すぎていないチャンクを選ぶ」方式

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---

---
## Q5 〔時系列･縦断〕数値把握／状況説明問題（スマートフォンの所有率の推移、利用状況の変化）

外部知識（『令和7年版 情報通信白書』第Ⅱ部 情報通信分野の現状と課題－第11節 デジタル活用の動向－1 国民生活におけるデジタル活用の動向－（1）情報通信機器・端末）に関する以下の文脈について、RAGシステムに問い合わせる：
デジタルを活用する際に必要となるインターネットなどに接続するための端末について、2024年の情報通信機器の世帯保有率は、「モバイル端末全体」で97.0％であり、その内数である「スマートフォン」は90.5％である。また、パソコンは66.4％となっている（図表Ⅱ-1-11-1）。



In [ ]:
# question
query = "これまでにスマートフォンの利用率はどのように推移してきましたか。また、スマートフォン利用状況はどのように変化してきましたか。"

* **これまでに**スマートフォンの**利用率（所有率）**はどのように推移してきましたか。また、スマートフォン利用状況はどのように変化してきましたか。
* **2025年までに** ～
* **2024年までに** ～  
⇒ 長期の縦断的・時系列の内容については、参照文書数を増やした方がいいか
---
* **2024年**におけるスマートフォンの**利用率／保有率**はどうなっていますか。（**令和7年（2025）版の情報通信白書**を参照して、回答して下さい）
* **2024年**におけるスマートフォンの**世帯保有率**は
* **2023年** ～  
⇒ どうしてこのような回答になるのか、原因不明！

In [ ]:
# question
query = "2024年におけるスマートフォンの利用率はどうなっていますか。令和7年（2025）版の情報通信白書を参照して、回答して下さい"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   Similarity RAG（cosine 類似度）：「質問と一番似ているチャンクを上から順に取ってくる」シンプルな方式
*   MMR RAG（MMR:Max Marginal Relevance）：「質問に近い＋相互に似すぎていないチャンクを選ぶ」方式

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---

---
## Q6 〔時系列･縦断〕概況･変化の説明問題（情報化の潮流･概況）

## **＜質問はこちらに＞**

In [ ]:
# question
query = "1973年から2025年までの50年ほどの情報化をめぐる潮流について、どのようにまとめることができますか。全体的な流れ・動向を述べてください。また、主たる潮流を5つ程度挙げ、その動向や変容についても述べてください。"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   Similarity RAG（cosine 類似度）：「質問と一番似ているチャンクを上から順に取ってくる」シンプルな方式
*   MMR RAG（MMR:Max Marginal Relevance）：「質問に近い＋相互に似すぎていないチャンクを選ぶ」方式

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---

---
## Q

外部知識（『？?年版 情報通信白書』？）に関する以下の文脈について、RAGシステムに問い合わせる：

>



---
## **＜質問はこちらに＞**

In [ ]:
# question
query = "　？"

## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   Similarity RAG（cosine 類似度）：「質問と一番似ているチャンクを上から順に取ってくる」シンプルな方式
*   MMR RAG（MMR:Max Marginal Relevance）：「質問に近い＋相互に似すぎていないチャンクを選ぶ」方式

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---

---
## Q

外部知識（『？?年版 情報通信白書』？）に関する以下の文脈について、RAGシステムに問い合わせる：

>



## **＜質問はこちらに＞**

In [ ]:
# question
query = "　？"

---
## ＜LLMの回答とRAGの回答の比較＞

### **RAGの回答**（Domain-specific answering）
*   Similarity RAG（cosine 類似度）：「質問と一番似ているチャンクを上から順に取ってくる」シンプルな方式
*   MMR RAG（MMR:Max Marginal Relevance）：「質問に近い＋相互に似すぎていないチャンクを選ぶ」方式

In [ ]:
# answer with the context from vector DB
anwser_sim = rag_chain_sim.invoke(query)
print("=== Similarity RAG ===")
display_answer(anwser_sim)

anwser_mmr = rag_chain_mmr.invoke(query)
print("\n=== MMR RAG ===")
display_answer(anwser_mmr)

### **LLMの回答**（Domain-general answering）

In [ ]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

---
## ＜ソース情報（Extracting sources）＞

### **RAGの回答のソースリスト**

In [ ]:
docs_sim = retriever_sim.invoke(query)
docs_mmr = retriever_mmr.invoke(query)

sim_retrieved_docs = retriever_sim.invoke(query)
sim_sources = format_sources(sim_retrieved_docs)

mmr_retrieved_docs = retriever_mmr.invoke(query)
mmr_sources = format_sources(mmr_retrieved_docs)


print("=== Similarity RAG ===")
print()
for d in docs_sim:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

print("\n=== MMR RAG ===")
print()
for d in docs_mmr:
    print(d.metadata.get("year"), d.metadata.get("title"))
    print(d.metadata.get("url"))
    print()

### **RAGの回答のソース詳細**

In [ ]:
# Display the sources separately
from IPython.display import HTML

print("=== Similarity Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(sim_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (Similarity):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

print("\n=== MMR Source metainfo (Metadata and Content) ===")
for i, doc in enumerate(mmr_retrieved_docs):
    url = doc.metadata.get('url', 'N/A')
    title = doc.metadata.get('title', 'N/A')
    year = doc.metadata.get('year', 'N/A')
    display(HTML(f"<b>Document {i+1} (MMR):</b><br>" +
                 f"Year: {year}<br>" +
                 f"Title: {title}<br>" +
                 f"URL: <a href='{url}' target='_blank'>{url}</a><br>" +
                 "<div style='word-wrap: break-word; overflow-wrap: break-word; white-space: pre-wrap;'>" +
                 f"Content: {doc.page_content}</div><br>"))

---